<a href="https://colab.research.google.com/github/flavianacif/DSWP/blob/master/RFB_RN_FINAL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tensorflow_addons as tfa
from tensorflow import keras
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from collections import Counter
from imblearn.over_sampling import SMOTE
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout


Verificar a versão do Tensorflow
> Assegurar que está a utilizar a versão 2.x.

In [ ]:
tf.__version__

Definir o número de casas decimais

In [ ]:
np.set_printoptions(precision= 3)

### 1. Carregar os dados

In [ ]:
# Leitura do dataframe:
df = pd.read_csv('/total_10.csv')

In [ ]:
df_total = df.copy()

In [ ]:
df_total.set_index('id', inplace=True)

In [ ]:
df_total.columns

### 2. Feature Selection

Foram retiradas as features abaixo, após análises e testes: 

*   rf2
*   cnae2 (porém cnae_secao permaneceu)
*   features MD e MC originais sem tratamento de outliers
*   features criadas com a soma das demais
*   'ind29', 'ind30', 'ind33', 'ind38', 'ind40', 'ind41': análise de que os valores unique ZEROS (ou UNS) representavam mais que 99,52%, sendo os demais valores irrelevantes

In [ ]:
df_total.drop(columns = ['rf2','cnae2','md1', 'md2', 'md3', 'md4', 'md5', 'md6', 'md7', 'md8',
       'md9', 'md10', 'md11', 'md12', 'mc1', 'mc2', 'mc3', 'mc4','mc_g1','ind_g1',
       'ind_g2','ind29', 'ind30','ind33', 'ind38', 'ind40', 'ind41'], inplace = True)

Features selecionadas para o modelo:

In [ ]:
df_total.columns

### 3. GET DUMMIES das categóricas

In [ ]:
df_total = pd.get_dummies(df_total)

### 4. Separação da base de Treinamento e Teste

In [ ]:
X_treino = df_total[df_total['target']!= 2]

In [ ]:
df_X = X_treino.copy()

In [ ]:
df_y = df_X['target']
df_X = df_X.drop(columns= ['target'])

In [ ]:
X_test = df_total[df_total['target']== 2].drop(columns= ['target'])

In [ ]:
X_treino.columns

### 5. Balanceamento da Amostra

In [ ]:
qtd = Counter(X_treino['target'])
qtd

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [ ]:
over = SMOTE(sampling_strategy = 0.5) # Reamostrar a classe minoritária para ter 50% da classe majoritária
under = RandomUnderSampler(sampling_strategy = 0.85) # Seleciona a classe majoritária para ter 85%
steps = [('over', over), ('under', under)]
pipeline = Pipeline(steps = steps)

In [ ]:
X, y = pipeline.fit_resample(df_X, df_y)

ANTES/DEPOIS:

In [ ]:
print(Counter(df_y),Counter(y))

### 6. Definir as amostras de treinamento e validação

Definir as amostras de treinamento e validação

In [ ]:
from sklearn.model_selection import train_test_split

X_treinamento, X_teste, y_treinamento, y_teste= train_test_split(X, y, test_size = 0.1, random_state = 22091980) #20111974)
print(f'X: Treinamento=  {X_treinamento.shape}; X: Teste=  {X_teste.shape}')

In [ ]:
print(f'Y: Treinamento =  {y_treinamento.shape}; Y: Teste = {y_teste.shape}')

### 7. Definir a arquitetura da Rede Neural com _Tensorflow_/_Keras_

Depois de testar várias possibilidades, chegamos na arquitetura abaixo, que foi responsável pelos nossos melhores resultados. Destaque para o número decrescente de neurônios nas camadas escondidas.

In [ ]:
# Número de Neurônios na Input Layer:
N_I = 72

# Número de neurônios na Output Layer:
N_O = 1

# Número de neurônios na Hidden Layer:
N_H = 400

N_H2 = 300

N_H3=200

N_H4 = 100

N_H5 = 50

N_H6 = 2


# Função de Ativação da Hidden Layer:
FA_H = tf.nn.leaky_relu

# Função de Ativação da Output Layer:
FA_O = tf.keras.activations.sigmoid

Definir as sementes para NumPy e Tensorflow:

In [ ]:
np.random.seed(19800922)
tf.random.set_seed(22091980)


### 8. Função Model() para definir e compilar a Rede Neural





* 6 camadas escondidas com _dropout_ com $p= 0.3$:
* optimizer= tf.keras.optimizers.Adam()
* loss= tf.keras.losses.BinaryCrossentropy()
* metrics= tfa.metrics.F1Score(1,average='macro',threshold=0.5)

In [ ]:
def Model():
  
  RN= Sequential()
  RN.add(Dense(N_H, input_dim= N_I, kernel_initializer= tf.keras.initializers.GlorotNormal(), activation= FA_H, kernel_constraint= tf.keras.constraints.UnitNorm()))
  RN.add(Dropout(0.3))
  RN.add(Dense(N_H2, kernel_initializer= tf.keras.initializers.GlorotNormal(), activation= FA_H, kernel_constraint= tf.keras.constraints.UnitNorm()))
  RN.add(Dropout(0.3))
  RN.add(Dense(N_H3, kernel_initializer= tf.keras.initializers.GlorotNormal(), activation= FA_H, kernel_constraint= tf.keras.constraints.UnitNorm()))
  RN.add(Dropout(0.3))
  RN.add(Dense(N_H4, kernel_initializer= tf.keras.initializers.GlorotNormal(), activation= FA_H, kernel_constraint= tf.keras.constraints.UnitNorm()))
  RN.add(Dropout(0.3))
  RN.add(Dense(N_H5, kernel_initializer= tf.keras.initializers.GlorotNormal(), activation= FA_H, kernel_constraint= tf.keras.constraints.UnitNorm()))
  RN.add(Dropout(0.3))
  RN.add(Dense(N_H6, kernel_initializer= tf.keras.initializers.GlorotNormal(), activation= FA_H, kernel_constraint= tf.keras.constraints.UnitNorm()))
  RN.add(Dropout(0.3))
  RN.add(Dense(units= N_O, activation= FA_O))
  
  Algoritmo_Opt = tf.keras.optimizers.Adam()
  Loss_Function = tf.keras.losses.BinaryCrossentropy()
  Metrics_Perf = tfa.metrics.F1Score(1,average='macro',threshold=0.5)

  RN.compile(optimizer = Algoritmo_Opt, loss = Loss_Function,metrics = Metrics_Perf)

  return RN


### 8. Fit da Rede Neural

A melhor epoch foi 57, com batch_size = 20. 

In [ ]:
np.random.seed(19800922)
tf.random.set_seed(22091980)
RN = Model()
hist= RN.fit(X_treinamento, y_treinamento, epochs = 57, batch_size = 20,
              validation_data = (X_teste, y_teste))

### 9. Avaliar a performance da Rede Neural

In [ ]:
def Mostra_ConfusionMatrix(X,y):
    y_pred = RN.predict_classes(X)
    mc = confusion_matrix(y, y_pred)
    sns.heatmap(mc/np.sum(mc), annot=True, annot_kws={"size": 10}, fmt='.2%', cmap='Blues')

X_teste

In [ ]:
RN.evaluate(X_teste, y_teste)

In [ ]:
F1_Test=f1_score(y_teste, RN.predict_classes(X_teste), average="macro")
print(f'F1: {F1_Test}')

In [ ]:
Mostra_ConfusionMatrix(X_teste, y_teste)

df_X: Base toda sem balanceamento

In [ ]:
RN.evaluate(df_X, df_y)

In [ ]:
F1_X=f1_score(df_y, RN.predict_classes(df_X), average="macro")
print(f'F1: {F1_X}')

In [ ]:
Mostra_ConfusionMatrix(df_X, df_y)

X_treinamento: base treinada, com balanceamento

In [ ]:
RN.evaluate(X_treinamento, y_treinamento)

In [ ]:
F1_Trein=f1_score(y_treinamento, RN.predict_classes(X_treinamento), average="macro")
print(f'F1: {F1_Trein}')

In [ ]:
Mostra_ConfusionMatrix(X_treinamento, y_treinamento)

### 10. Fazer Predições com a Rede Neural

In [ ]:
y_test = RN.predict_classes(X_test)

In [ ]:
df_submit = pd.concat([pd.DataFrame(X_test.index,columns=['id']),pd.DataFrame(y_test,columns=['target']).astype('boolean')],axis=1)

In [ ]:
df_submit.head()

In [ ]:
df_submit['target'].value_counts()

In [ ]:
F = df_submit['target'].value_counts()[0]
T = df_submit['target'].value_counts()[1]

In [ ]:
#n=11 #número da tentativa
n = n + 1

### 11. Geração do arquivo csv para submissão

In [ ]:
df_submit.to_csv(f'/PyLadies_NL_{n}_Test_{round(F1_Test,4)}_X_{round(F1_X,4)}_Trein_{round(F1_Trein,4)}_F{F}_T{T}.csv',index = False, sep = ',')

### 12. Salvar a Rede Neural

In [ ]:
# Save the weights
RN.save(f'/RN_{n}.h5')

### 13. Loop com epochs de 1 a 100: FIT, PREDICT, gera arquivo para submissão contendo os scores e salva a Rede Neural
Foi feito um loop com epochs de 1 a 100. Os melhores resultados foram utilizados no Voting que gerou o resultado final.

In [ ]:
for i in range(1,101):
  np.random.seed(19800922)
  tf.random.set_seed(22091980)
  RN = Model()
  hist= RN.fit(X_treinamento, y_treinamento, epochs = i, batch_size = 20,
              validation_data = (X_teste, y_teste))

  F1_Test=f1_score(y_teste, RN.predict_classes(X_teste), average="macro")
  F1_X=f1_score(df_y, RN.predict_classes(df_X), average="macro")
  F1_Trein=f1_score(y_treinamento, RN.predict_classes(X_treinamento), average="macro")
    
  y_test = RN.predict_classes(X_test)

  df_submit = pd.concat([pd.DataFrame(X_test.index,columns=['id']),pd.DataFrame(y_test,columns=['target']).astype('boolean')],axis=1)

  F = df_submit['target'].value_counts()[0]
  T = df_submit['target'].value_counts()[1]

  df_submit.to_csv(f'PyLadies_NL_{n}.{i}_Test_{round(F1_Test,4)}_X_{round(F1_X,4)}_Trein_{round(F1_Trein,4)}_F{F}_T{T}.csv',index = False, sep = ',')
  RN.save(f'RN_{n}.{i}.h5')


### 14. Salva o Notebook

In [ ]:
%notebook RFB_RN_{n}.ipynb